In [19]:
#### Customer support chatbot using Mistral API and FAISS
## Define categories and keywords
import os
from dotenv import load_dotenv


In [20]:
## Initialize Mistral client
from mistralai import Mistral

from dotenv import find_dotenv


def load_environment_variables():
    find_dotenv('Machine learning_projects/.env')
    MISTRAL_API = os.getenv("MISTRAL_API")
    client = Mistral(api_key=MISTRAL_API)
    return MISTRAL_API, client


In [21]:
#creating the state class
class ChatbotState:
    def __init__(self):
        self.category = None
        self.user_query = None
        self.context = None

In [22]:
load_dotenv()

CATEGORIES = {
    "Billing": ["invoice", "payment", "charged", "refund"],
    "Technical Support": ["error", "problem", "not working", "crash", "issue", "bug"],
    "General Inquiry": ["contact", "hours", "location", "address", "help"]
} 

In [23]:
## Function to categorize user queries
def categorize_query(query):
    for category, keywords in CATEGORIES.items():
        if any(keyword in query.lower() for keyword in keywords):
            return category
    return "Unknown"
## Function to get response based on category
RESPONSES = {
    "Billing": "For billing questions, please check your account settings or contact our billing support team.",
    "Technical Support": "For technical support, please try to restart your device or describe your issue in detail and our technical support team will assist you.",
    "General Inquiry": "For general inquiries, please visit our FAQ page or contact our general support team. Our business hours are 9 AM to 5 PM, Monday to Friday.",
    "Unknown": "I'm sorry, I couldn't categorize your query. Please provide more details as I connect you with a support agent."
}


In [24]:
# define mistral_client
MISTRAL_API, mistral_client = load_environment_variables()

In [25]:
## Function to generate response
def generate_response(query, category):
    if category in RESPONSES:
        return RESPONSES[category]
    
    # If category is unknown, use RAG to generate a response
    relevant_chunks = mistral_client.similarity_search(query, k=3)
    context = "\n".join([chunk.page_content for chunk in relevant_chunks])
    
    prompt = f"Context: {context}\n\nQuestion: {query}\nAnswer:"
    
    response = mistral_client.chat.completions.create(
        model="mistral-7b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful customer support assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    
    return response.choices[0].message['content']

In [26]:
#Function to run customer support
def Query(user_query):
    # Load environment variables and initialize Mistral client
    MISTRAL_API, client = load_environment_variables()
    
    # Initialize chatbot state
    state = ChatbotState()
    state.user_query = user_query
    
    # Categorize the user query
    state.category = categorize_query(state.user_query)
    
    # Generate response based on category
    RESPONSES = generate_response(state.user_query, state.category)
    

    return RESPONSES

In [11]:
%pip install -U sentence-transformers

  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import faiss
import os

# Sample documents
documents = [
    "Your invoice is available in your account settings.",
    "You can make a payment using credit card or PayPal.",
    "If you were incorrectly charged, please contact support for a refund.",
    "For technical issues, try restarting your device.",
    "If the app crashes, please report the bug to our technical team.",
    "For general inquiries, visit our FAQ page or contact support.",
    "Our business hours are 9 AM to 5 PM, Monday to Friday."
]

# Wrap documents in LangChain Document objects
docs = [Document(page_content=doc) for doc in documents]

# Create embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vector_store = FAISS.from_documents(docs, embedding_model)

# Perform a similarity search
query = "I have an issue with my invoice and payment."
results = vector_store.similarity_search(query)

# Print results
for i, res in enumerate(results):
    print(f"{i+1}. {res.page_content}")

1. Your invoice is available in your account settings.
2. If you were incorrectly charged, please contact support for a refund.
3. You can make a payment using credit card or PayPal.
4. For technical issues, try restarting your device.


In [28]:
## Ask a question
user_query = "I have an issue with my invoice and payment."
response = Query(user_query)
print("Response:", response)

Response: For billing questions, please check your account settings or contact our billing support team.


In [29]:
# Set user interface
from zipfile import PyZipFile
import gradio as gr

def respond_to_query(user_query):
    response = Query(user_query)
    return response

iface = gr.Interface(fn=respond_to_query, inputs="text", outputs="text")

In [30]:
def rag_query(user_query):
    # Retrieve relevant documents from FAISS
    results = vector_store.similarity_search(user_query, k=3)
    
    # Combine retrieved content
    context = "\n".join([doc.page_content for doc in results])
    
    # Generate a response using Mistral or another LLM
    prompt = f"Context:\n{context}\n\nQuestion: {user_query}\nAnswer:"
    response = mistral_client.chat(prompt=prompt)
    
    return response

In [ ]:
import gradio as gr
demo = gr.Interface(
    fn=rag_query,
    inputs=gr.Textbox(lines=5, placeholder="Ask a question about customer support..."),
    outputs=gr.Textbox(lines=10),
    title="Customer Support Chatbot"
)
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "c:\Users\user\anaconda3\Lib\site-packages\gradio\queueing.py", line 745, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\gradio\blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\gradio\blocks.py", line 1623, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\user\anaconda3\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
      